### Case - [Pierre-Antoine Denarié]

### TL;DR
ADD description

In [ ]:
# Imports
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os

In [ ]:
# Data imports
data_folder = "Data"
for file in os.listdir(data_folder):
    if file.endswith(".csv"):
        df_name = os.path.splitext(file)[0]
        globals()[df_name] = pd.read_csv(os.path.join(data_folder, file))